In [1]:
import requests

In [2]:
SIZE = 16

In [3]:
class Mouse:
    def __init__(self, token="", url="http://127.0.0.1:8801/api/v1"):
        self.token = token
        self.url = url
        self.maze = [[0 for _ in range(SIZE)] for _ in range(SIZE)]
        self.data: dict
        self.x = 15
        self.y = 0
        self.direction = "up"

    def forward(self):
        r = requests.post(self.url + "/robot-cells/forward", params={"token": self.token})
        if r.status_code != requests.codes.ok:
            raise RuntimeError
        if self.direction == "up":
            self.x -= 1
        elif self.direction == "right":
            self.y += 1
        elif self.direction == "left":
            self.y -= 1
        elif self.direction == "down":
            self.x += 1

    def turn_left(self):
        r = requests.post(self.url + "/robot-cells/left", params={"token": self.token})
        if r.status_code != requests.codes.ok:
            raise RuntimeError
        next_d = {
            "up":    "left",
            "right": "up",
            "left":  "down",
            "down":  "right",
        }
        self.direction = next_d[self.direction]

    def turn_right(self):
        r = requests.post(self.url + "/robot-cells/right", params={"token": self.token})
        if r.status_code != requests.codes.ok:
            raise RuntimeError
        next_d = {
            "up":    "right",
            "right": "down",
            "left":  "up",
            "down":  "left",
        }
        self.direction = next_d[self.direction]

    def backward(self):
        r = requests.post(self.url + "/robot-cells/backward", params={"token": self.token})
        if r.status_code != requests.codes.ok:
            raise RuntimeError
        if self.direction == "up":
            self.x += 1
        elif self.direction == "right":
            self.y -= 1
        elif self.direction == "left":
            self.y += 1
        elif self.direction == "down":
            self.x -= 1

    def go(self, direction: str):
        if direction == "up":
            if self.direction == "up":
                self.forward()
                return
            if self.direction == "right":
                self.turn_left()
                self.forward()
                return
            if self.direction == "left":
                self.turn_right()
                self.forward()
                return
            if self.direction == "down":
                self.turn_right()
                self.turn_right()
                self.forward()
                return
        if direction == "right":
            if self.direction == "up":
                self.turn_right()
                self.forward()
                return
            if self.direction == "right":
                self.forward()
                return
            if self.direction == "left":
                self.turn_right()
                self.turn_right()
                self.forward()
                return
            if self.direction == "down":
                self.turn_left()
                self.forward()
                return
        if direction == "left":
            if self.direction == "up":
                self.turn_left()
                self.forward()
                return
            if self.direction == "right":
                self.turn_right()
                self.turn_right()
                self.forward()
                return
            if self.direction == "left":
                self.forward()
                return
            if self.direction == "down":
                self.turn_right()
                self.forward()
                return
        if direction == "down":
            if self.direction == "up":
                self.turn_right()
                self.turn_right()
                self.forward()
                return
            if self.direction == "right":
                self.turn_right()
                self.forward()
                return
            if self.direction == "left":
                self.turn_left()
                self.forward()
                return
            if self.direction == "down":
                self.forward()
                return

    def upd_sensor_data(self):
        r = requests.get(self.url + "/robot-cells/sensor-data", params={"token": self.token})
        if r.status_code != requests.codes.ok:
            raise RuntimeError
        self.data = r.json()

    def restart(self):
        r = requests.post(self.url + "/robot-cells/restart", params={"token": self.token})
        if r.status_code != requests.codes.ok:
            raise RuntimeError

    @staticmethod
    def __dist_to_wall(dist: int):
        return int(round(dist / 166.67))

    def get_walls(self):
        f = self.__dist_to_wall(self.data["front_distance"])
        r = self.__dist_to_wall(self.data["right_side_distance"])
        l = self.__dist_to_wall(self.data["left_side_distance"])
        b = self.__dist_to_wall(self.data["back_distance"])
        return f, r, l, b

    def get_abs_walls(self):
        f, r, l, b = m.get_walls()
        # True if there is wall
        # False if we can pass through
        up = not ((m.direction == "up" and f != 0) or (m.direction == "right" and l != 0) or (m.direction == "down" and b != 0) or (m.direction == "left" and r != 0))
        right = not ((m.direction == "up" and r != 0) or (m.direction == "right" and f != 0) or (m.direction == "down" and l != 0) or (m.direction == "left" and b != 0))
        down = not ((m.direction == "up" and b != 0) or (m.direction == "right" and r != 0) or (m.direction == "down" and f != 0) or (m.direction == "left" and l != 0))
        left = not ((m.direction == "up" and l != 0) or (m.direction == "right" and b != 0) or (m.direction == "down" and r != 0) or (m.direction == "left" and f != 0))
        return up, right, left, down

    def fill_cell(self):
        walls = {
            #  (up, right, left, down)
            (False, False, False, False): 0,
            (False, False, True,  False): 1,
            (True,  False, False, False): 2,
            (False, True,  False, False): 3,
            (False, False, False, True ): 4,
            (False, False, True,  True ): 5,
            (False, True,  False, True ): 6,
            (True,  True,  False, False): 7,
            (True,  False, True,  False): 8,
            (False, True,  True,  False): 9,
            (True,  False, False, True ): 10,
            (True,  True,  False, True ): 11,
            (True,  True,  True,  False): 12,
            (True,  False, True,  True ): 13,
            (False, True,  True,  True ): 14,
            (True,  True,  True,  True ): 15,
        }
        self.maze[self.x][self.y] = walls[*self.get_abs_walls()]

In [4]:
def fill_maze(m: Mouse, visited):
    m.upd_sensor_data()
    m.fill_cell()
    visited[m.x][m.y] = True
    up, right, left, down = m.get_abs_walls()
    if up and not visited[m.x - 1][m.y]:
        m.go("up")
        fill_maze(m, visited)
        m.go("down")
    if right and not visited[m.x][m.y + 1]:
        m.go("right")
        fill_maze(m, visited)
        m.go("left")
    if left and not visited[m.x][m.y - 1]:
        m.go("left")
        fill_maze(m, visited)
        m.go("right")
    if down and not visited[m.x + 1][m.y]:
        m.go("down")
        fill_maze(m, visited)
        m.go("up")

In [172]:
m = Mouse()
fill_maze(m, [[False for _ in range(SIZE)] for _ in range(SIZE)])

IndexError: list index out of range